#### Setup


In [1]:
import torch
import torchvision
from lib.data_handlers import Load_ImageNet100, Load_PACS
from overcomplete.models import DinoV2, ViT, ResNet, ViT_Large, SigLIP
from torch.utils.data import DataLoader, TensorDataset
from overcomplete.sae import TopKSAE, train_sae
from overcomplete.visualization import (overlay_top_heatmaps, evidence_top_images, zoom_top_images, contour_top_image)
import os
import matplotlib.pyplot as plt
from einops import rearrange
from lib.universal_trainer import train_usae
from lib.activation_generator import Load_activation_dataloader
import torch.nn as nn
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR
from lib.eval import evaluate_models
from lib.visualizer import visualize_concepts
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

### Train a USAE

Define Models and Generate Their Activations

In [2]:
models = {
    "Dino": DinoV2(device="cuda"),
    "ViT": ViT(device="cuda")
}

activation_dir = "activations/ImageNet100_Dino_ViT"
image_loader = Load_ImageNet100(transform=None, batch_size=256, shuffle=True)


activations_dataloader = Load_activation_dataloader(
    models=models,
    image_dataloader=image_loader,
    max_seq_len=196,   
    save_dir=activation_dir, 
    generate=False,  
    rearrange_string='n t d -> (n t) d'
    )

Using cache found in C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
c:\Users\sproj_ha\miniconda3\envs\interpretability\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SAE Training

In [3]:
concepts = 768 * 8
epochs = 50
lr=3e-4
sample = next(iter(activations_dataloader))

In [4]:
SAEs = {}
optimizers = {}
schedulers = {}

for key, model in models.items():

    SAEs[key] = TopKSAE(
        sample[f"activations_{key}"].shape[-1],
        nb_concepts=concepts,
        top_k=16,
        device="cuda",
    )
    optimizers[key] = torch.optim.Adam(SAEs[key].parameters(), lr=lr)

    # Set up a Linear + Cosine Scheduler
    warmup_scheduler = LinearLR(
        optimizers[key], start_factor=1e-6 / 3e-4, end_factor=1.0, total_iters=10
    )
    cosine_scheduler = CosineAnnealingLR(optimizers[key], T_max=epochs, eta_min=1e-6)
    schedulers[key] = SequentialLR(
        optimizers[key],
        schedulers=[warmup_scheduler, cosine_scheduler],
        milestones=[25],
    )

criterion = nn.L1Loss(reduction="mean")  # change to mean reduction

In [5]:
train_usae(
    names=list(models.keys()),
    models=SAEs,
    dataloader=activations_dataloader,
    criterion=criterion,
    nb_epochs=epochs,
    optimizers=optimizers,
    schedulers=schedulers,
    device="cuda",
    seed=42,
)

Epoch 1/50:   9%|▉         | 48/508 [00:23<02:49,  2.72it/s, loss=1.65]c:\Users\sproj_ha\miniconda3\envs\interpretability\Lib\site-packages\torch\optim\lr_scheduler.py:198: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
Epoch 1/50: 100%|██████████| 508/508 [03:20<00:00,  2.54it/s, loss=1.33]



[Epoch 1] Loss: 763.1322 | Time: 200.33s | Dead Features: 0.0%


Epoch 2/50: 100%|██████████| 508/508 [03:02<00:00,  2.79it/s, loss=1.23]



[Epoch 2] Loss: 650.4296 | Time: 182.31s | Dead Features: 0.0%


Epoch 3/50: 100%|██████████| 508/508 [02:58<00:00,  2.85it/s, loss=1.2] 


...


[Epoch 48] Loss: 528.9405 | Time: 183.22s | Dead Features: 26.2%


Epoch 49/50: 100%|██████████| 508/508 [03:04<00:00,  2.75it/s, loss=1.07] 



[Epoch 49] Loss: 528.6036 | Time: 184.78s | Dead Features: 26.0%


Epoch 50/50: 100%|██████████| 508/508 [03:04<00:00,  2.76it/s, loss=1.05] 


[Epoch 50] Loss: 528.1778 | Time: 184.30s | Dead Features: 26.3%


In [6]:
model_state_dicts = {name: model.state_dict() for name, model in SAEs.items()}
model_path = "./models/USAE_Dino_ViT"
torch.save(model_state_dicts, model_path)

In [5]:
model_path = "./models/USAE_Dino_ViT"
state = torch.load(model_path)

for name, sae in SAEs.items():
    print(sae.load_state_dict(state[name]))

<All keys matched successfully>
<All keys matched successfully>


In [6]:
val_activation_dir = "activations/ImageNet100_Dino_ViT_VAL"
val_image_loader = Load_ImageNet100(transform=None, batch_size=256, shuffle=True, train=False)

val_activations_dataloader = Load_activation_dataloader(
    models=models,
    image_dataloader=val_image_loader,
    max_seq_len=196,   
    save_dir=val_activation_dir, 
    generate=False,  
    rearrange_string='n t d -> (n t) d'
)

In [7]:
visualize_concepts(
    activation_loader=val_activations_dataloader,
    SAEs=SAEs,
    num_concepts=concepts,
    n_images=8,
    patch_width=14, 
    save_dir="results/visualizer2_Dino_ViT/",
    abort_threshold=1.0,
)

Generating Activations


100%|██████████| 20/20 [01:12<00:00,  3.64s/it]


Saving Concepts


100%|██████████| 6144/6144 [31:20<00:00,  3.27it/s]  


In [ ]:
import matplotlib.pyplot as plt
import torch
from einops import rearrange
import heapq
from overcomplete.visualization.plot_utils import (
    get_image_dimensions,
    interpolate_cv2,
    show,
)
from overcomplete.visualization.cmaps import VIRIDIS_ALPHA
import os
import itertools
from pathlib import Path
from multiprocessing import Pool


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def save_concept_plot(concept_id, save_dir):
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists

    filename = f"concept_{concept_id:04d}.png"
    filepath = save_dir / filename
    plt.savefig(filepath, bbox_inches="tight", dpi=300)


def plot_concept_grid(concept_id, heaps, SAEs, n_images, save_dir, counter):
    """
    Plots and saves the visualization grid for a single concept.
    """
    rows_per_model = 2
    total_rows = len(SAEs) * rows_per_model
    fig = plt.figure(figsize=(n_images * 2.5, total_rows * 2.5))

    for j, (name, _) in enumerate(SAEs.items()):
        heap = heaps[name][concept_id]
        # fill_heap_with_black(heap, n_images, counter)
        # top_items = get_top_items_from_heap(heap, n_images)

        title_row = j * rows_per_model + 1
        image_row_start = title_row + 1

        # Title row
        title_ax = plt.subplot(total_rows, n_images, (title_row - 1) * n_images + 1)
        plt.axis("off")
        plt.text(
            0.5,
            0.5,
            name,
            ha="center",
            va="center",
            fontsize=14,
            weight="bold",
            transform=title_ax.transAxes,
        )

        # Image rows
        for i in range(n_images):

            if i > len(heap):
                image = torch.zeros((3, 224, 224))
                ax = plt.subplot(total_rows, n_images, idx)
                ax.axis("off")
                show(image)
            
            else:
                idx = (image_row_start - 1) * n_images + i + 1
                image = heap[i][2]["image"]
                heat = heap[i][2]["heatmap"]
                width, height = get_image_dimensions(image)
                heatmap = interpolate_cv2(heat, (width, height))
                ax = plt.subplot(total_rows, n_images, idx)
                ax.axis("off")
                show(image)
                show(heatmap, cmap=VIRIDIS_ALPHA, alpha=1.0)

    plt.tight_layout()
    save_concept_plot(concept_id, save_dir)
    plt.close()



def visualize_concepts(
    activation_loader,
    SAEs,
    save_dir,
    patch_width,
    num_concepts,
    n_images=10,
    abort_threshold=0,
):
    # Initialize a heap for each model that will store the (score, image) for ONE model
    heaps = {}
    heaps = {
        name: {concept_id: [] for concept_id in range(num_concepts)}
        for name in SAEs.keys()
    }

    # Initialize Tie Breaker on Heaps
    counter = itertools.count()

    #### OPTIMIZED VISUALIZER
    for batch in tqdm(activation_loader):

        for name, sae in SAEs.items():

            # Encode only once
            with torch.no_grad():
                _, heatmaps = sae.encode(
                    batch[f"activations_{name}"].squeeze().to(device)
                )
                heatmaps = rearrange(
                    heatmaps, "(n w h) d -> n w h d", w=patch_width, h=patch_width
                )  # shape: [B, w, h, D]

            B, w, h, D = heatmaps.shape

            # Compute mean heatmap per image per concept: [B, D]
            mean_heatmaps = heatmaps.mean(dim=(1, 2))  # shape: [B, D]

            # Get top-k per concept efficiently using torch.topk
            # indices: [k, D], values: [k, D]
            topk_values, topk_indices = torch.topk(mean_heatmaps.T, k=n_images, dim=1)

            for concept_id in range(D):

                for rank in range(n_images):
                    idx = topk_indices[concept_id, rank].item()
                    score = topk_values[concept_id, rank].item()

                    # Only proceed if this score is better than the worst in the heap, or heap not full AND score is not 0
                    heap = heaps[name][concept_id]
                    if (
                        len(heap) < n_images or -score > heap[0][0]
                    ) and score > abort_threshold:
                        z = {
                            "image": batch["images"].squeeze()[idx].cpu(),
                            "heatmap": heatmaps[idx, :, :, concept_id].cpu(),
                        }
                        score_item = (-score, next(counter), z)

                        if len(heap) < n_images:
                            heapq.heappush(heap, score_item)
                        else:
                            heapq.heappushpop(heap, score_item)

    for concept in tqdm(range(num_concepts)):
        if all(len(heaps[name][concept]) == 0 for name in SAEs.keys()):
            continue  # DO NOT VISUALIZE CONCEPT IF ITS EMPTY FOR ALL MODELS
        else:
            plot_concept_grid(concept, heaps, SAEs, n_images, save_dir, counter)



In [10]:
visualize_concepts(
    activation_loader=val_activations_dataloader,
    SAEs=SAEs,
    num_concepts=concepts,
    n_images=8,
    patch_width=14, 
    save_dir="results/visualizer_Dino_ViT/",
    abort_threshold=0.0,
)

 65%|██████▌   | 4000/6144 [46:42<25:02,  1.43it/s]  


MemoryError: bad allocation

MemoryError: Unable to allocate 1.53 MiB for an array with shape (224, 224, 4) and data type float64

<Figure size 2000x1000 with 18 Axes>

In [ ]:
### SAVE THE Z_sAES and then on them
